In [1]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [2]:
from datetime import datetime as dt
import pytz

In [3]:
scopes = ['https://www.googleapis.com/auth/calendar']

In [4]:
flow = InstalledAppFlow.from_client_secrets_file("masha_credentials.json", scopes=scopes)

In [8]:
credentials = flow.run_console()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=358671830953-l2k1p6h7ggf001gnd7hpgamnsephnut7.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=eUOyTV32Z6tzLpTPK9onXRsZch8BqU&prompt=consent&access_type=offline
Enter the authorization code: 4/4QFpjTsr5V5yQbOyXXcQCgZ_Iet25ZGxPMa4JjxeSXwq7X8X90KhMJA


In [6]:
import pickle

In [5]:
pickle.dump(credentials, open("token.pkl", "wb"))

NameError: name 'pickle' is not defined

In [7]:
#next time
credentials = pickle.load(open("token.pkl", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'token.pkl'

In [9]:
service = build("calendar", "v3", credentials=credentials)

In [10]:
cal_result = service.calendarList().list().execute()

In [ ]:
cal_result['items']

In [11]:
cnt = 0
calendar_list = []
for i in cal_result['items']:
    calendar_list.append(i['summary'])
for i in calendar_list:
    print(cnt, i)
    cnt +=1
calendar_choice = int(input("\nWhich of these calendars would you like to use? (number) "))

0 masha.geo@gmail.com
1 NYT Astronomy and Space Calendar
2 Classes
3 Summer
4 Family
5 HackMIT 2020
6 https://registrar.mit.edu/calendar-ical/current/93/all/calendar.ics
7 Contacts
8 Holidays in United States
9 Праздники РФ
10 Weather

Which of these calendars would you like to use? (number) 2


In [12]:
def get_calendar_list():
    cnt = 0
    calendar_list = []
    for i in cal_result['items']:
        calendar_list.append(str(cnt)+' '+i['summary'])
        cnt +=1
    return calendar_list

In [ ]:
test_cal_list = get_calendar_list()

In [ ]:
test_cal_list

In [13]:
calendar_id = cal_result['items'][calendar_choice]['id'] #my Classes calendar

In [ ]:
#test_cal_id = cal_result['items'][calendar_choice]['id']

In [ ]:
#type(cal)

In [14]:
result = service.events().list(calendarId=calendar_id).execute()

In [ ]:
#testing
print(type(result['items'][0]))
result['items'][0]
#so every class in the calendar is a dictionary w each of the tags as an entry

In [ ]:
#get the school timezone
cont = input("What continent? ").title() 
#need to make a note of the choices. lowk can we just have them choose from a list lmao
#bc like America vs North America vs Americas
city = input("What city? ").title().replace(" ", "_")
school_tz = cont+'/'+city

In [ ]:
# want to update every event in calendar to have a timezone tag bc the user might not have done that
# - or do we just assume that they did that??
# either way i'll put this in later

### event = service.events().get(calendarId=calendar_id, eventId='eventId').execute()
### updated_event = service.events().update(calendarId=calendar_id, eventId=event['id'], body=event).execute()



In [15]:
import copy

In [16]:
#create a dictionary where each entry is class_name:{dic of its atributes from user input}
user_dic = {}
#create a dictionary where each entry is class_name:{event params from gcal import minus event id}
event_dic = {}
for course in result['items']:
    print(course['summary'])
    sync=int(input("synchronous? (0,1,2) ")) #0=asynch, 1=recorded but maybe pref to watch live, 2=synch
    user_dic[course['summary']]={'sync':sync}
    event_dic[course['summary']]= copy.deepcopy(course) 
    try:
        del event_dic[course['summary']]['id']#remove the event id bc that's specific to this calendar and i just need the event parameters
        del event_dic[course['summary']]['htmlLink']
        del event_dic[course['summary']]['iCalUID']
        del event_dic[course['summary']]['etag']
        del event_dic[course['summary']]['organizer']
        del event_dic[course['summary']]['recurringEventId']
    except KeyError:
        pass
    

4.A01
synchronous? (0,1,2) 2
6.0001 lec
synchronous? (0,1,2) 1
B1.2 Deutsch
synchronous? (0,1,2) 2
6.0001 rec
synchronous? (0,1,2) 1
11.S03
synchronous? (0,1,2) 2
8.022 rec
synchronous? (0,1,2) 2
18.03 lec
synchronous? (0,1,2) 0
18.03 rec
synchronous? (0,1,2) 2
Tutmonda
synchronous? (0,1,2) 2
Tutmonda
synchronous? (0,1,2) 2
6.0001 rec
synchronous? (0,1,2) 1
4.001
synchronous? (0,1,2) 2
21G.058
synchronous? (0,1,2) 2
21G.058
synchronous? (0,1,2) 2
8.022 lec
synchronous? (0,1,2) 0
8.022 lec
synchronous? (0,1,2) 0


In [17]:
event_dic

{'4.A01': {'kind': 'calendar#event',
  'status': 'confirmed',
  'created': '2020-08-31T17:19:27.000Z',
  'updated': '2020-08-31T17:37:37.424Z',
  'summary': '4.A01',
  'description': 'https://mit.zoom.us/j/97833456008?pwd=TEs2NUI5SGFCOCtKWHpGVDZ4TkQ0dz09',
  'colorId': '2',
  'creator': {'email': 'masha.geo@gmail.com'},
  'start': {'dateTime': '2020-09-14T12:00:00-04:00',
   'timeZone': 'America/New_York'},
  'end': {'dateTime': '2020-09-14T13:00:00-04:00',
   'timeZone': 'America/New_York'},
  'recurrence': ['RRULE:FREQ=WEEKLY;WKST=SU;UNTIL=20201215T045959Z;BYDAY=MO'],
  'sequence': 0,
  'reminders': {'useDefault': True}},
 '6.0001 lec': {'kind': 'calendar#event',
  'status': 'confirmed',
  'created': '2020-09-01T02:16:23.000Z',
  'updated': '2020-09-01T02:17:04.649Z',
  'summary': '6.0001 lec',
  'description': 'https://mit.zoom.us/j/94826488181',
  'colorId': '3',
  'creator': {'email': 'masha.geo@gmail.com'},
  'start': {'dateTime': '2020-09-02T15:00:00-04:00',
   'timeZone': 'Amer

In [21]:
#create a new calendar, and then we'll add events to it directly
home_tz = pytz.timezone('Europe/Moscow') #switch to user input later
newcalendar = {'summary': 'new_test_calendar','timeZone': 'Europe/Moscow'} #need to change cal name lol

created_calendar = service.calendars().insert(body=newcalendar).execute()

new_calendar_id = created_calendar['id']

In [22]:
school_tz = pytz.timezone('America/New_York')

In [23]:
#first, sort the events by synchronicity
synch = [course[0] for course in user_dic.items() if course[1]['sync']==2]
semisynch = [course[0] for course in user_dic.items() if course[1]['sync']==1]
asynch = [course[0] for course in user_dic.items() if course[1]['sync']==0]
 

In [27]:
def freebusy():
    dt1 = home_tz.localize(dt(2020, 9, 21, 12))
    dt2 = home_tz.localize(dt(2020, 9, 22, 12))
    body = {"kind":"calendar#freeBusy",
      "timeMin": dt1.isoformat(), #need tp have it do next monday to next sunday
      "timeMax": dt2.isoformat(),
      "timeZone": 'Europe/Moscow',
      "items": [
        {
          "id": new_calendar_id
        }
      ]
    }

    free = service.freebusy().query(body=body).execute()
    return free

In [25]:
#first, create the events for the completely synchronous classes

for name in synch:
    print(name)
    service.events().insert(calendarId=new_calendar_id, body=event_dic[name]).execute()


4.A01
B1.2 Deutsch
11.S03
8.022 rec
18.03 rec
Tutmonda
4.001
21G.058


In [ ]:
def force_freebusy():
    try:
        free = freebusy()
    except HttpError:
        try:
            free = freebusy()
        except HttpError:
            try:
                free = freebusy()
            except HttpError:
                free = freebusy()


In [28]:
free = freebusy()

In [29]:
sleep_time = int(input("How many hours would you like to sleep? "))
time_of_day = int(input("Are you a morning person (0) or a night owl (1)? (enter 0 or 1) "))

How many hours would you like to sleep? 7
Are you a morning person (0) or a night owl (1)? (enter 0 or 1) 0


In [ ]:
free

In [30]:
#next, set a sleep time 

times_list = []
class_starts = []
class_ends = []
for dic in free['calendars'][new_calendar_id]['busy']:
    print(dic)
    #get the intervals as datetime
    times_list.append(dic['start'])
    class_starts.append(dic['start'])
    times_list.append(dic['end'])
    class_ends.append(dic['end'])
    
times_list.sort()
times_list = [dt.strptime(time[0:-6],'%Y-%m-%dT%H:%M:%S') for time in times_list]
#class_starts = [dt.strptime(time[0:-6],'%Y-%m-%dT%H:%M:%S') for time in class_starts]
#class_ends = [dt.strptime(time[0:-6],'%Y-%m-%dT%H:%M:%S') for time in class_ends]
class_starts = [int(time[11:13]) for time in class_starts]
class_ends = [int(time[11:13]) for time in class_ends]

try:
    latest_class = max(i for i in class_ends if i <= 5) 
except:
    latest_class = max(class_ends)

earliest_class = min(i for i in class_starts if i >= 5)


def make_sleep_event():
    event = { #need to add recurrence!
          'summary': 'Sleept',
          'description': 'zzzzzz', 
        'recurrence': ['RRULE:FREQ=DAILY'],
          'start': {
            'dateTime': start_datetime.isoformat(),
            'timeZone': 'Europe/Moscow',
          },
          'end': {
            'dateTime': stop_datetime.isoformat(),
            'timeZone': 'Europe/Moscow',
          }
        }
    event = service.events().insert(calendarId=new_calendar_id, body=event).execute()
       
if time_of_day == 0: #if morning person
    #find the sleep start time that's 10 pm or 1 hr after your last class, whichever is earlier
    if latest_class <= 21 and latest_class >= 12:
        #sleep at 22
        start_datetime = dt(2020, 9, 21, 22)
        stop_datetime = dt(2020, 9, 22, (22+sleep_time)%24)
        make_sleep_event()
    else:
        #sleep at latest_class+1
        if latest_class <= 5:
            start_datetime = dt(2020, 9, 22, latest_class+1) #this would be so much easier achieved w datetimes ngl
        else:
            start_datetime = dt(2020, 9, 21, latest_class+1)             
        stop_datetime = dt(2020, 9, 22, (latest_class+1+sleep_time)%24)
        make_sleep_event()
else: #if night owl
    #find the sleep start that ends at 12 pm or an hour before earliest class, whichever is later
    if earliest_class >= 13:
        #sleep until 12
        start_datetime = dt(2020, 9, 22, 12-sleep_time)
        stop_datetime = dt(2020, 9, 22, 12)
        make_sleep_event()
    else:
        start_datetime = dt(2020, 9, 22, earliest_class-1-sleep_time)
        stop_datetime = dt(2020, 9, 22, earliest_class-1)
        make_sleep_event()
    


{'start': '2020-09-21T19:00:00+03:00', 'end': '2020-09-21T20:00:00+03:00'}
{'start': '2020-09-21T21:00:00+03:00', 'end': '2020-09-21T22:00:00+03:00'}


In [ ]:
#test
#dt.strptime(time[0:-6],'%Y-%m-%dT%H:%M:%S')
dt.strptime('2020-09-21T19:00:00+03:00'[0:-6],'%Y-%m-%dT%H:%M:%S')

In [ ]:
free

In [ ]:
#now, check if you can make the half synch (1) classes work using freebusy method

#free = freebusy()


In [32]:
#schedule as many as you can to be synch

for name in semisynch:
    print(name)
    print(event_dic[name]['start'], event_dic[name]['end'])
    start_time = str(event_dic[name]['start']['dateTime'])[11:16]
    end_time = str(event_dic[name]['end']['dateTime'])[11:16]
    
    free = {}
    we_good = True
    for i in range(21,22):
        #test that yuo're not busy any day at that time
        dt1 = school_tz.localize(dt(2020, 9, i, int(start_time[0:2]), int(start_time[3:5])))
        dt2 = school_tz.localize(dt(2020, 9, i, int(end_time[0:2]), int(end_time[3:5])))
        print(dt1,dt2)
        body = {"kind":"calendar#freeBusy",
          "timeMin": dt1.isoformat(), #need tp have it do next monday to next sunday
          "timeMax": dt2.isoformat(),
          "timeZone": 'America/New_York',
          "items": [
            {
              "id": new_calendar_id
            }
          ]
        }

        free = service.freebusy().query(body=body).execute()
        
        if not free['calendars'][new_calendar_id]['busy']==[]:
            asynch.append(name)
            we_good = False
            break
    
    if we_good:
        service.events().insert(calendarId=new_calendar_id, body=event_dic[name]).execute()


6.0001 lec
{'dateTime': '2020-09-02T15:00:00-04:00', 'timeZone': 'America/New_York'} {'dateTime': '2020-09-02T16:30:00-04:00', 'timeZone': 'America/New_York'}
2020-09-21 15:00:00-04:00 2020-09-21 16:30:00-04:00
6.0001 rec
{'dateTime': '2020-09-18T11:00:00-04:00', 'timeZone': 'America/New_York'} {'dateTime': '2020-09-18T12:00:00-04:00', 'timeZone': 'America/New_York'}
2020-09-21 11:00:00-04:00 2020-09-21 12:00:00-04:00


In [ ]:
#finally, schedule in the rest of the asynch classes
asynch.append('8.022 lec')


In [ ]:
asynch

In [ ]:
#test
(datetime.datetime(2020, 9, 21, 21).isoformat())

In [ ]:
free['calendars'][new_calendar_id]['busy']